## Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### 1. What test is appropriate for this problem? Does CLT apply? ###
The problem statement is about two groups and their proportion of success vs. failures. It is therefore appropriate to use significance testing given the *difference* of two proportions.

And yes, central limit theorem would apply. As long as we have a discrete distribution, even if its shape does not resemble a normal curve at all, when we take samples of the distribution and when we average the samples and plot them, we'll arrive at a normal curve no matter what.

### 2. What are the null and alternate hypotheses? ###
* **Null Hypothesis**: $Proportion_{white\space callback} = Proportion_{black\space callback}$
* **Alternative Hypothesis**: $Proportion_{white\space callback} \neq Proportion_{black\space callback}$

We can interpret the null hypothesis as the difference between the two *success* proportions is zero.

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches. ###

We assign our critical value `alpha` as 0.05 so if our p-value is less than this, we'd reject the null hypothesis. The length of the sampled dataframes and the probability of success for both groups are specified below.

In [6]:
import scipy.stats as stats

alpha = 0.05

w_size = len(w)
b_size = len(b)

w_success = len(w[w["call"] == 1]) / w_size
b_success = len(b[b["call"] == 1]) / b_size

print("Critical value: %.2f" % alpha)
print("Proportion of white group getting callback: %.2f%%" \
      % (w_success*100))
print("Proportion of black group getting callback: %.2f%%" \
      % (b_success*100))

Critical value: 0.05
Proportion of white group getting callback: 9.65%
Proportion of black group getting callback: 6.45%


Let's first take the frequentist approach. We first calculate the pooled sample proportion `pooled_prop`. From there, the standard error `SE` can be derived by mutliplying the probability of sucess with the probability of failure and dividing it by the standard deviation of the sample population. Finally we get the *z-statistic* by simply getting the difference of the success proportions of both groups and dividing them by the standard error.

In [7]:
pooled_prop = (w_success*w_size + b_success*b_size) \
                / (w_size + b_size)

SE = (pooled_prop * (1-pooled_prop) \
     * ((1/w_size) + (1/b_size)))**0.5

z = (w_success - b_success) / SE

print("Pooled sample proportion: %.3f%%" % pooled_prop)
print("Standard Error: %.3f%%" % SE)
print("Z-statistic: %.3f%%" % z)

Pooled sample proportion: 0.080%
Standard Error: 0.008%
Z-statistic: 4.108%


We calculated the above elements just by using formulas but in order for us to get a precise p-value we'll use scipy stats instead of a z-table.

We see that the p-value turns out to be much less than our critical value. And so we cannot accept the null hypothesis. This means that unfortunately, there *is* a significant difference between the successes of white applicants and of black applicants.

In [8]:
p = stats.norm.sf(abs(z))

print("P-value: %.5f\n" % p)
print("P-Value larger than alpha? %s" % str(p > alpha))
if p > alpha:
    print("Proportion of the two groups are NOT different")
else:
    print("Proportion of the two groups ARE different")

P-value: 0.00002

P-Value larger than alpha? False
Proportion of the two groups ARE different


Let's take a step forward and see how much difference the two groups have. From the simple calculation below, we arrive at `3.2%` as our `prop_diff`.

In [9]:
prop_diff = w_success - b_success

print("Proportion difference between two groups: %.1f%%" \
      % (prop_diff*100))

Proportion difference between two groups: 3.2%


But this is just from *one* trial. Who's to say that what we've gotten is actually on the fringe of the norm? And so we proceed to getting our confidence interval so we have a feel of where the true proportional difference lies.

We'll use `95%` as our confidence level, so as we extend outward from the distribution mean, we get two borders that contain 95% of the curve area. The `distance` below is the cumulative distribution proportion up to that *right* border. We multiply this by the standard deviation of the difference between the proportions. This is the `std_dev_diff` below.

Our confidence interval then is `0.032` plus or minus `0.0153`.

This then tells us that if we've repeated the experiment several times, we'd see in 95% of them that the *true* proportion difference of the two groups is between `1.68%` and `4.73%`.

In [10]:
conf_lev = 0.95

distance = stats.norm.ppf((1+conf_lev)/2)
std_dev_diff = ((w_success *(1-w_success))/w_size \
                + (b_success *(1-b_success))/b_size)**0.5
conf_int = distance * std_dev_diff

print("Confidence Interval: %.3f\n" % conf_int)
print("True proportion difference is between %.2f%% and %.2f%%." \
      % ((prop_diff - conf_int)*100, (prop_diff + conf_int)*100))

Confidence Interval: 0.015

True proportion difference is between 1.68% and 4.73%.


Now let's see if we can reproduce the same conclusion by taking the bootstrap approach. Again `prop_diff` is the difference between the proportion of successes of both groups. In order for us to test our hypothesis, we'll assume truth in the null hypothesis by performing permutation on the original `data` dataframe 10,000 times. We'll actually shuffle the index of the dataframe then classify the first half as the `w` group and the next as the `b` group. From there, we take the mean of each permutated group. The difference of the mean per loop is then added onto the `simulation` array.

The p-value `p` is just the proportion of the instances that each `simulation` element happens to be greater than or equal to the `prop_diff`. This didn't happen at all in the 10,000 trials so our p-value is `0`. It is therefore incredibly rare to see such difference between successful callbacks from white-sounding names and those from black-sounding names **if** it were purely up to chance. Clearly, there is a factor of bias then.

This is the exact same conclusion we got from the frequentist approach.

In [11]:
prop_diff = w_success - b_success

np.random.seed(42)
simulation = []
for i in range(10000):
    rand_ind = np.random.randint(0, len(data), len(data))
    perm_data = data["call"].iloc[rand_ind]
    perm_w = perm_data[:len(w)]
    perm_b = perm_data[len(b):]
    perm_w_mean = np.mean(perm_w)
    perm_b_mean = np.mean(perm_b)
    simulation.append(perm_w_mean - perm_b_mean)
simulation = np.array(simulation)

p = np.sum(simulation >= prop_diff)/10000
print("P-value: %.5f\n" % p)
print("P-Value larger than alpha? %s" % str(p > alpha))
if p > alpha:
    print("Proportion of the two groups are NOT different")
else:
    print("Proportion of the two groups ARE different")

P-value: 0.00000

P-Value larger than alpha? False
Proportion of the two groups ARE different


Let's now see what *could* be the actual value for the difference between the two proportions using confidence intervals. To get the 95% confidence bounds, we simply take the 2.5th and 97.5th percentile of our `simulation` distribution. The area between these borders are 95%.

Like with what we've seen in the previous approach, we're 95% confident that the *true* proportion difference of the two groups is between `1.64%` and `4.72%`.

In [12]:
conf_int = np.percentile(simulation, [2.5, 97.5])

print("Confidence Interval: %.3f\n" % conf_int[1])
print("True proportion difference is between %.2f%% and %.2f%%." \
      % ((prop_diff + conf_int[0])*100, (prop_diff + conf_int[1])*100))

Confidence Interval: 0.015

True proportion difference is between 1.64% and 4.72%.


### 4. Write a story describing the statistical significance in the context or the original problem. ###
The proportion of applicants with white-sounding names that get callbacks is 9.65%. For applicants with black-sounding names, the rate is 6.45%. That's a mean difference of 3.2%, which, from what we've calculated, *is* significant. In a simulation of 10,000 trials, not one had that much disparity when we've assumed no difference between the groups. So the numbers tell us that there is evidence of discrimination in the hiring process (from unconscious bias, perhaps), that not even a random process can justify.

### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis? 
One might say that perhaps race did not play much of a factor in the experiment given that more weight is naturally given to education, experience, etc. But as I re-read the problem statement, it was indicated that names were *randomly* assigned to the resumes. Below, I made a quick check to see how much difference there are between those given with white names and those given with black names *in terms of* honors, education, and experience. As the report mentioned, it appears to be random indeed. So the only factor that's differentiating the two groups is race alone. So, yes, this only further approves our statistical finding that there *is* discrimination.

In [13]:
honors = np.mean(w["honors"]) - np.mean(b["honors"])
education = np.mean(w["education"]) - np.mean(b["education"])
experience = np.mean(w["yearsexp"]) - np.mean(b["yearsexp"])

print("Difference in `honors`: %.2f%%" \
      % (honors*100))
print("Difference in `education`: %.2f%%" \
      % (education*100))
print("Difference in `experience`: %.2f%%" \
      % (experience*100))

Difference in `honors`: 0.29%
Difference in `education`: 0.49%
Difference in `experience`: 2.67%
